# Using CTIPe with Kamodo on AWS server


## Instructions from R. Ringuette

**Instructions for calling CTIPe data from fortran (slow method – via file output)**   


The instructions for calling CTIPe data from fortran are outlined below. Instructions and notes are indicated by the bulleted points, while commands are given with the dash-style bullets. In the current state, the python scripts are specifically designed for the Geodyn software and the CTIPe model, but can be extended to other models and even more easily to other software calls, depending upon the requirements. These directions enable a slow interface between fortran and python through a result file, which is projected to take roughly four hours to execute for a 2-week arc (15 min cadence). *A faster interface is under investigation via ForPy.*




### (Before running)  Create a conda environment with Kamodo

1. Create conda environment with kamodo installed. 

   1. ``cd (to dir where you want things to run) ``
   2. ``cp (full path)/Kamodo-Master ``
   3. ``conda create -n FortranKamodo python=3.7.9 ``
   4. ``conda install -n FortranKamodo -c conda-forge plotly sympy scipy pytest pandas hydra-core requests ipython``
   5. ``conda activate FortranKamodo``  
   6. ``pip install python-forge``  
   7.  ``pip install netCDF4``  
   8.  ``pip install ./Kamodo-Master``  
   9.  ``conda deactivate (when done)``  
 
**Note**:  Use the Kamodo-Master dir RR sent to avoid possible issues with new versions.

2. Can test your kamodo installation by executing the following commands with the environment activated (in command line):
    1.  ``ipython``
    2. ``import kamodo``
    3. If no error occurs, your installation is good. You may exit ipython (``exit()``) and continue.


### Prepare the CTIPe Data:

From RR--- "The below code is used to speed up the data processing in the kamodo reader. Don’t worry about the other file types from the CTIPe output. The data wrapper function takes care of it."


In [1]:
# from kamodo.readers import ctipe_data_wrapper as DW
# import glob
# ##### copy over the directory name where the data files are stored
# # file_dir = '/data/geodyn_proj/interface_kamodo_geodyn/GeoDynTest_Slow/'
# file_dir = '/data/data_geodyn/atmos_models_data/ctipe/2018_Dec_1_15/' 

# files = glob.glob(file_dir + '*plot-density.nc')

# for f in files: 
#     DW.ctipe_wrap_files(f)


In [2]:
# files

### Check satellite flythrough functionality

From RR--- "With the conda environment activated, execute the following command in the conda command prompt:"


``python C:\Users\rringuet\Kamodo_WinDev1\Kamodo-master\kamodo\readers\CTIPe_wrapper_fortrancmd.py file_dir rho ilev
1426637500.0 400.0 -25.0 10.0``

   - `File_dir` is the directory where the data is located  
   - `rho` is the variable name   
   - `ilev` means the variable depends on the pressure level for the CTIPe model.   
   - `sat_time` is the satellite timestamp in UTC since Jan 1 1970   
   - `sat_height`: is satellite altitude above the ground (in km)  
   - `sat_lat` satellite latitude  
   - `sat_lon` satellite longitude  
    
  

From RR---  
- This program finds the value of the density and the density derivative at the time and location specified for the data files in the given directory. The two values are printed in a simple file called results.txt in the same directory as the data files, with the two values separated by a comma. The density, rho (kg/m^3), is given first, and rho_dz is given second, both printed to the 15 th decimal place.  
- Note: The results.txt file is overwritten each time the program is run.
- If the results.txt file is produced without error, then you can continue.



In [3]:
# """
# Created on Tue Mar 30 15:21:50 2021
# @author: rringuet
# Adapted for speed to execute for a single point 
# """
# #coding imports
# import numpy as np
# import glob, sys
# from datetime import datetime, timedelta, timezone
# #from kamodo.readers.ctipe_fast import CTIPe as CTIPe_fast
# #from kamodo.readers.ctipe_time import CTIPe, ctipe_varnames
# from kamodo.readers.ctipe_faster_wrapped import CTIPe, ctipe_varnames 

# #need input times to be timestamps since 1970 to pull data from the correct files
# #modified to reduce number of times the ctipe reader is called to reduce calc time
   
# def CTIPeVariables():
#     '''return a list of all the possible variables in CTIPe'''
    
#     return ctipe_varnames

# def CTIPe_Single_Prerun(file_dir, variable_list):
#     '''Return a list of the required height input for each variable'''

#     #check for wrapped data output in dir, create if necessary
#     wrapped_files = glob.glob(file_dir+'*-plot-density-wrapped.nc')
#     files = glob.glob(file_dir+'*-plot-density.nc')
#     if len(wrapped_files)!=len(files):  #not all files are wrapped, if any
#         print('Wrapped files not found. Generating...')
#         from kamodo.readers.ctipe_data_wrapper import ctipe_wrap_files as wrap
#         filename = [wrap(f) for f in files if f.split('.nc')[0]+'-wrapped.nc' not in wrapped_files][0]
#         #produce the wrapped files for all not in file_dir, returns the new filenames, takes first                
#     else:
#         filename = wrapped_files[0]
    
#     #create ctipe object, return vertical dependencies for variables requested
#     ctipe = CTIPe(filename, variables_requested=variable_list, printfiles=False)
#     var_test = []
#     if 'H' in variable_list: variable_list.remove('H')  #H only needed if other functions require ilev
#     for var in variable_list:  #determine which variables require ilev(2), height(1), neither(0)
#         input_var_list = ctipe.variables[var]['xvec']
#         if 'ilev' in input_var_list.keys(): var_test.append('ilev')
#         elif 'height' in input_var_list.keys(): var_test.append('height')
#         else: var_test.append('none')
    
#     return var_test

# def ts_to_hrs(time_val, filedate):
#     '''Convert array of timestamps to hours since midnight of filedate string'''
    
#     file_datetime = datetime.strptime(filedate+' 00:00:00', '%Y-%m-%d %H:%M:%S')
#     return (datetime.utcfromtimestamp(time_val)-file_datetime).total_seconds()/3600.

# def hrs_to_ts(time_val, filedate):
#     '''Convert array of hours since midnight of filedate string to timestamps'''
    
#     file_datetime = datetime.strptime(filedate+' 00:00:00', '%Y-%m-%d %H:%M:%S').replace(tzinfo=timezone.utc)    
#     return datetime.timestamp(file_datetime+timedelta(hours=time_val))

# sample_ilev = np.linspace(1,15,75,dtype=float)   #global variable
# def CalcIlev(H, t, height, lat, lon):
#     '''Approximate ilev by inverting the gridded height function CTIPe.H for one sat point'''
#     print('t',t )
#     print('height',height )
#     print('lat',lat )
#     print('lon',lon )

    
#     rough_height = H(np.array([[t, ilev, lat, lon] for ilev in sample_ilev]))
#     print('rough_height', rough_height*1e-3)

#     ilev_range = np.sort(sample_ilev[np.argsort(abs(height-rough_height))[0:2]])
#     test_ilev = np.linspace(ilev_range[0],ilev_range[1],100,dtype=float)
#     finer_height = H(np.array([[t, ilev, lat, lon] for ilev in test_ilev]))
#     print('finer_height', finer_height*1e-3)
#     print('return CalcIlev: ',test_ilev[np.argmin(abs(height-finer_height))])
#     return test_ilev[np.argmin(abs(height-finer_height))]

# def test_ctipevalid(ctipe, sat_time):
#     ''' Determine if a new ctipe object is needed bsed on the time given'''
    
#     if isinstance(ctipe, list):  
#         return True  #if a list, then ctipe DNE, get a new one
#     elif (sat_time>=ctipe.filetimes[0]) and (sat_time<=ctipe.filetimes[1]):
#         return False  #sat_time is within known time range of file, use current ctipe
#     else:  #sat_time is not within known time range of file, get a new ctipe object
#         return True

# def new_ctipe_object(file_dir, variable_list, sat_time):
#     ''' Return a new ctipe object valid for the given time'''
    
#     files, times = glob.glob(file_dir+'*-plot-density-wrapped.nc'), []
#     for f in files:
#         file_date = f.split('\\')[-1].split('/')[-1][:10]
#         beg_ts = hrs_to_ts(0.25, file_date)  #ctipe data starts at 0.25
#         end_ts = hrs_to_ts(24.0, file_date)  #and ends at 24.00
#         times.extend([beg_ts,end_ts])
#         if (sat_time>=beg_ts) and (sat_time<=end_ts):
#             print(f'New CTIPe object is from {f}')
#             return CTIPe(f, variables_requested=variable_list, printfiles=False) #fast
#     return times
    

# def CTIPe_Single_FlyAway(ctipe, variable_list, sat_time, sat_height, sat_lat, sat_lon, 
#                   z_dependence=['none'], dz=''):
#     '''fly satellite through CTIPe model data, per position
#     sat_time, sat_height, sat_lat, and sat_lon are all floats, not arrays
#     z_dependence = ["none","height","ilev"] if variables depend on all three options
#        dependence must be in same order as variable_list to match with variable names'''
    
        
#     #Create satellite tracks with appropriate inputs
#     file_date = ctipe.datetimes[0][0:10]
#     model_sat_time = ts_to_hrs(sat_time, file_date)
#     if 'H' in variable_list: variable_list.remove('H')  #H only needed if other functions require ilev
#     sat_track = {}  #initialize list of satellite tracks
#     if "none" in z_dependence:
# #         print('z_dependence == none')
#         sat_track['none']=[model_sat_time,sat_lat,sat_lon]
#     if "height" in z_dependence:  #if function requires height (in km)
# #         print('z_dependence == height')
#         sat_track['height']=[model_sat_time,sat_height/1000.,sat_lat,sat_lon]
#     if "ilev" in z_dependence:  #if ilev is required for at least one variable
# #         print('z_dependence == ilev')
#         sat_ilev = CalcIlev(ctipe.H, *[model_sat_time,sat_height*1000.,sat_lat,sat_lon])  
        
#         sat_track['ilev']=[model_sat_time,sat_ilev,sat_lat,sat_lon]

#     #retrieve interpolator and interpolate data for each variable. 
#     results = {variable_list[i]: ctipe[variable_list[i]](sat_track[z_dependence[i]])[0] for i
#                in range(len(variable_list))}
    
#     #determine vertical derivatives for each variable if requested
#     if dz!='':
#         for i in range(len(variable_list)):
#             if dz[i] and z_dependence[i]!='none':  #if dz requested and variable has a vertical dependence
#                 #generate tracks with slightly larger and smaller heights
#                 if z_dependence[i]=='height':
#                     #stay within CTIPe height (km) boundaries
#                     sat_height_low = sat_height/1000.-100
#                     if sat_height_low <= 140.: sat_height_low = 140.
#                     sat_height_high = sat_height/1000.+100.
#                     if sat_height_high>=2000.: sat_height_high = 2000.
#                     dz_track = [[model_sat_time,sat_height_low,sat_lat,sat_lon],
#                                 [model_sat_time,sat_height_high,sat_lat,sat_lon]]
#                 if z_dependence[i]=='ilev':
#                     #stay within CTIPe ilev boundaries
#                     sat_ilev_low = sat_ilev-1.
#                     if sat_ilev_low<=1.: sat_ilev_low = 1.
#                     sat_ilev_high = sat_ilev+1.
#                     if sat_ilev_high >= 15.: sat_ilev_high = 15.
#                     dz_track =  [[model_sat_time,sat_ilev_low,sat_lat,sat_lon],
#                                 [model_sat_time,sat_ilev_high,sat_lat,sat_lon]]
#                 dz_result = ctipe[variable_list[i]](dz_track)  #returns two values
#                 results[variable_list[i]+'_dz'] = dz_result[1]-dz_result[0]
#     return results

# def find_singlefile(file_pattern, sat_time, reader, dt=450., verbose=False):
#     '''Find file containing given single time. Adjust if within dt seconds.'''
    
#     #build lookup table
#     files, times, ts_list, filename = glob.glob(file_pattern), {}, [], ''
#     for f in files:
#         k = reader(f, variables_requested=[], filetimes=True)
#         file_date = k.datetimes[0][0:10]
#         times[file_date] = [f,k.timerange['min'],k.timerange['max'],
#                             k.filetimes[0], k.filetimes[1]]
#         ts_list.extend([k.filetimes[0], k.filetimes[1]])
    
#     #choose file time is in
#     for file_date in times.keys():
#         print(file_date)
#         if ((sat_time>=times[file_date][3]) & (sat_time<=times[file_date][4])):
#             filename = times[file_date][0]
#             break
#     if filename=='':  #if not in a file, try closest file if within dt
#         sat_time = [ts_list[abs(np.array(ts_list)-sat_time).argmin()] if \
#                           (abs(np.array(ts_list)-sat_time).min() < dt) else 0][0]
#         if sat_time==0:  #error if not within dt
#             raise AttributeError('Files in dir do not contain the given time.')
#         elif verbose:
#             print(f'Adjusting time to nearest file with {dt} seconds.')
#         for file_date in times.keys():  #otherwise, find file
#             if ((sat_time>=times[file_date][3]) & (sat_time<=times[file_date][4])):
#                 filename = times[file_date][0]
#                 break 
            
#     return filename, sat_time



In [4]:
# 

In [5]:

# kamodo_path     = '/data/geodyn_proj/interface_kamodo_geodyn/Kamodo-master/kamodo/readers/CTIPe_wrapper_fortrancmd.py'
# model_data_path = '/data/data_geodyn/atmos_models_data/ctipe/2015_03_18/'
# # model_data_path = '/data/data_geodyn/atmos_models_data/ctipe/2018_Dec_1_15/'
# var_inputs_static= 'rho ilev'

# time = float('1426637500.0')# '150318 121010'    #'181201 210519'#'1426637500.0'
# alt  = float('400.0') #'485.4546760656302808'
# lat  = float('-25.0') # '40.18117715905883'
# lon  = float('10.0') #'-117.07021273480323'


# file_dir      = model_data_path
# sat_time      = time
# sat_height    = alt
# sat_lat       = lat
# sat_lon       = lon
# variable_list = ['rho']
# z_dependence  = ['ilev']

In [6]:


# #find file nearest sat_time (within 450 seconds), correct sat_time if needed
# filename, sat_time = find_singlefile(file_dir+'*plot-density-wrapped.nc', 
#                                      sat_time, CTIPe, dt=450., verbose=False)

# #get ctipe object for requested variable (+H too)
# # this just "Registers" the model as kamodo object
# ctipe = CTIPe(filename, variables_requested=variable_list, printfiles=False)

# # print(ctipe)
# #get results requested for single position given
# results_dict = CTIPe_Single_FlyAway(ctipe, variable_list, 
#                                 sat_time, sat_height, sat_lat, sat_lon, 
#                                 z_dependence=z_dependence, 
#                                 dz=[1])
# print(results_dict)

In [7]:
# !pip install sympy==1.7.1


In [8]:
import os

# command = 'python /data/geodyn_proj/interface_kamodo_geodyn/Kamodo-master/kamodo/readers/CTIPe_wrapper_fortrancmd.py /data/data_geodyn/atmos_models_data/ctipe/2018_Dec_1_15/ rho ilev 181201 210519 485.4546760656302808    40.18117715905883  -117.07021273480323'

kamodo_path     = '/data/geodyn_proj/interface_kamodo_geodyn/Kamodo-master/kamodo/readers/CTIPe_wrapper_fortrancmd.py'
model_data_path = '/data/data_geodyn/atmos_models_data/ctipe/2015_03_18/'
# model_data_path = '/data/data_geodyn/atmos_models_data/ctipe/2018_Dec_1_15/'
var_inputs_static= 'rho ilev'

time = '150318 121010'    #'181201 210519'#'1426637500.0'
alt  = '400.0' #'485.4546760656302808'
lat  = '-25.0' # '40.18117715905883'
lon  = '10.0' #'-117.07021273480323'
  
# rho ilev 181201 210519 485.4546760656302808    40.18117715905883  -117.07021273480323'

command = ('python'          +' '+
           kamodo_path       +' '+
           model_data_path   +' '+
           var_inputs_static +' '+
           time              +' '+
           alt               +' '+
           lat               +' '+
           lon)
os.system(command)

with open(model_data_path+'results.txt', 'r') as f:
    for line_no, line in enumerate(f):
        print(line)


0.000000000003996

-0.000000000009748


In [9]:
command

'python /data/geodyn_proj/interface_kamodo_geodyn/Kamodo-master/kamodo/readers/CTIPe_wrapper_fortrancmd.py /data/data_geodyn/atmos_models_data/ctipe/2015_03_18/ rho ilev 150318 121010 400.0 -25.0 10.0'

In [10]:
# 1426712719.0

In [11]:
import datetime as dt  # Python standard library datetime  module
import numpy as np
from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/


def ncdump(path , verb=True):
    
    nc_fid = Dataset(path)

    
    '''
    ncdump outputs dimensions, variables and their attribute information.
    The information is similar to that of NCAR's ncdump utility.
    ncdump requires a valid instance of Dataset.

    Parameters
    ----------
    nc_fid : netCDF4.Dataset
        A netCDF4 dateset object
    verb : Boolean
        whether or not nc_attrs, nc_dims, and nc_vars are printed

    Returns
    -------
    nc_attrs : list
        A Python list of the NetCDF file global attributes
    nc_dims : list
        A Python list of the NetCDF file dimensions
    nc_vars : list
        A Python list of the NetCDF file variables
    '''
    def print_ncattr(key):
        """
        Prints the NetCDF file attributes for a given key

        Parameters
        ----------
        key : unicode
            a valid netCDF4.Dataset.variables key
        """
        try:
            print("\t\ttype:", repr(nc_fid.variables[key].dtype))
            for ncattr in nc_fid.variables[key].ncattrs():
                print('\t\t%s:' % ncattr)
                repr(nc_fid.variables[key].getncattr(ncattr))
        except KeyError:
            print("\t\tWARNING: %s does not contain variable attributes" % key)

    # NetCDF global attributes
    nc_attrs = nc_fid.ncattrs()
    if verb:
        print("NetCDF Global Attributes:")
        for nc_attr in nc_attrs:
            print('\t%s:' % nc_attr, repr(nc_fid.getncattr(nc_attr)))
    nc_dims = [dim for dim in nc_fid.dimensions]  # list of nc dimensions
    # Dimension shape information.
    if verb:
        print("NetCDF dimension information:")
        for dim in nc_dims:
            print("\tName:", dim )
            print("\t\tsize:", len(nc_fid.dimensions[dim]))
            print_ncattr(dim)
    # Variable information.
    nc_vars = [var for var in nc_fid.variables]  # list of nc variables
    if verb:
        print("NetCDF variable information:")
        for var in nc_vars:
            if var not in nc_dims:
                print('\tName:', var)
                print("\t\tdimensions:", nc_fid.variables[var].dimensions)
                print("\t\tsize:", nc_fid.variables[var].size)
                print_ncattr(var)
    return(nc_attrs, nc_dims, nc_vars)




#     dataset = Dataset('../data/TIEGCM_CCMC/my_stuff/s_002.nc')
#     #dataset = Dataset('../data/TIEGCM_CCMC/s010.nc')

#     ncdump(dataset)


#     data = Dataset('../data/TIEGCM_CCMC/my_stuff/s_002.nc')
#     lon = np.array(data.variables['lon'])
#     lat = np.array(data.variables['lat'])
#     print lon
#     print lat

In [12]:
file_dir = '/data/data_geodyn/atmos_models_data/ctipe/2018_Dec_1_15/' 
# file_dir = '/data/data_geodyn/atmos_models_data/ctipe/2015_03_18/' 

file = file_dir+'2018-12-03-plot-height.nc'
# file = file_dir+'2015-03-18-plot-density-wrapped.nc'

# ncdump(file , verb=True)

In [13]:
from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/

nc_fid = Dataset(file)
nc_fid

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    dimensions(sizes): time(96), ht(94), lat(91), lon(20)
    variables(dimensions): float64 time(time), float32 ht(ht), float32 lat(lat), float32 lon(lon), float32 electron_density(time, ht, lat, lon), float32 atomic_oxygen_ion_density(time, ht, lat, lon), float32 atomic_hydrogen_ion_density(time, ht, lat, lon), float32 ion_temperature(time, ht, lat, lon), float32 electron_temperature(time, ht, lat, lon)
    groups: 

In [14]:
nc_fid.variables.keys()

dict_keys(['time', 'ht', 'lat', 'lon', 'electron_density', 'atomic_oxygen_ion_density', 'atomic_hydrogen_ion_density', 'ion_temperature', 'electron_temperature'])

In [15]:
nc_fid.variables['ht'][:]

masked_array(data=[ 140.,  160.,  180.,  200.,  220.,  240.,  260.,  280.,
                    300.,  320.,  340.,  360.,  380.,  400.,  420.,  440.,
                    460.,  480.,  500.,  520.,  540.,  560.,  580.,  600.,
                    620.,  640.,  660.,  680.,  700.,  720.,  740.,  760.,
                    780.,  800.,  820.,  840.,  860.,  880.,  900.,  920.,
                    940.,  960.,  980., 1000., 1020., 1040., 1060., 1080.,
                   1100., 1120., 1140., 1160., 1180., 1200., 1220., 1240.,
                   1260., 1280., 1300., 1320., 1340., 1360., 1380., 1400.,
                   1420., 1440., 1460., 1480., 1500., 1520., 1540., 1560.,
                   1580., 1600., 1620., 1640., 1660., 1680., 1700., 1720.,
                   1740., 1760., 1780., 1800., 1820., 1840., 1860., 1880.,
                   1900., 1920., 1940., 1960., 1980., 2000.],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [16]:
# np.max(nc_fid.variables['ht'][:, :, :, :]*1e-3)